# 第8章: 機械学習

本章では，Bo Pang氏とLillian Lee氏が公開しているMovie Review Dataのsentence polarity dataset v1.0を用い，文を肯定的（ポジティブ）もしくは否定的（ネガティブ）に分類するタスク（極性分析）に取り組む．

In [2]:
!head  rt-polaritydata/rt-polaritydata/rt-polarity.neg

simplistic , silly and tedious . 
it's so laddish and juvenile , only teenage boys could possibly find it funny . 
exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 
[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . 
a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . 
the story is also as unoriginal as they come , already having been recycled more times than i'd care to count . 
about the only thing to give the movie points for is bravado -- to take an entirely stale concept and push it through the audience's meat grinder one more time . 
not so much farcical as sour . 
unfortunately the story and the actors are served with a hack script . 
all the more disquieting for its relatively gore-free allusions to the serial murders , but it falls down in its attempts to humaniz

## 70. データの入手・整形
文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

1. rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
2. rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
3. 上述1と2の内容を結合（concatenate）し，行をランダムに並び替える

sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [90]:
input1 ="rt-polarity.pos"
input2="rt-polarity.neg"
output="sentiment.txt"

words = []
with open(input1, 'r') as pos:
    words.extend(['+1 {}'.format(line.strip()) for line in pos])
with open(input2, 'r') as neg:
    words.extend(['+1 {}'.format(line.strip()) for line in neg])
import random
random.shuffle(words)

with open(output, 'w') as file_out:
    print(*words, sep='\n', file=file_out)

In [98]:
input1 ="rt-polarity.neg" #5332
input2="rt-polarity.pos" #5331
#input1 = 'rt-polaritydata/rt-polaritydata/rt-polarity.pos'
#input2 = 'rt-polaritydata/rt-polaritydata/rt-polarity.neg'
output="sentiment.txt"
f = open(output,"w")
words=[]
for line in open(input1):
    words.append("-1 "+ line)
    #writelines("-1 "+line)
print("neg>",len(words))
for line in open(input2):
    words.append("+1 "+ line)
    #writelines("+1 "+line)
    
import random
random.shuffle(words)
#出力
for line in words:
    print(line,end="",file=f)

neg> 5331


In [94]:
import codecs
import random

fname_pos = 'rt-polaritydata/rt-polaritydata/rt-polarity.pos'
fname_neg = 'rt-polaritydata/rt-polaritydata/rt-polarity.neg'
fname_smt = 'sentiment.txt'
fencoding = 'cp1252'        # Windows-1252らしい

result = []

# ポジティブデータの読み込み
with codecs.open(fname_pos, 'r', fencoding) as file_pos:
    result.extend(['+1 {}'.format(line.strip()) for line in file_pos])

# ネガティブデータの読み込み
with codecs.open(fname_neg, 'r', fencoding) as file_neg:
    result.extend(['-1 {}'.format(line.strip()) for line in file_neg])

# シャッフル
random.shuffle(result)

# 書き出し
with codecs.open(fname_smt, 'w') as file_out:
    print(*result, sep='\n', file=file_out)

In [67]:
!head -n 10 sentiment.txt

+1 it is different from others in its genre in that it is does not rely on dumb gags , anatomical humor , or character cliches ; it primarily relies on character to tell its story . 
-1 painfully padded . 
+1 a wild comedy that could only spring from the demented mind of the writer of being john malkovich . 
+1 overall , it's a very entertaining , thought-provoking film with a simple message : god is love . 
-1 heavy with flabby rolls of typical toback machinations . 
+1 this rich , bittersweet israeli documentary , about the life of song-and-dance-man pasach'ke burstein and his family , transcends ethnic lines . 
-1 unless there are zoning ordinances to protect your community from the dullest science fiction , impostor is opening today at a theater near you . 
+1 the socio-histo-political treatise is told in earnest strides . . . [and] personal illusion is deconstructed with poignancy . 
-1 this long and relentlessly saccharine film is a clear case of preaching to the converted . 
+1 

In [78]:
!wc rt-polarity.neg
!wc rt-polarity.pos
!wc sentiment.txt

    5331  111563  612259 rt-polarity.neg
    5330  112384  626146 rt-polarity.pos
   10609  233470 1264182 sentiment.txt


In [84]:
!cat rt-polarity.pos rt-polarity.neg > sentiment.txt
!wc sentiment.txt

   10661  223947 1238405 sentiment.txt


In [82]:
pos_count,neg_count =0,0
for line in open('sentiment.txt'):
    if line[0:2]=="+1":
        pos_count+=1
    elif line[0:2] =="-1":
        neg_count+=1
    else:
        print(line)
print(pos_count,neg_count)

5310 5299


## 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

【stop word】あまりにたくさん検索にかかりすぎるので、検索精度の向上のためには検索対象から除外せざるを得ない語。助詞や助動詞などの機能語(日本語ならば「は」「の」「です」「ます」など、英語ならば「the」「of」「is」など)は、殆んどの場合これに該当する。

In [134]:
from collections import Counter
words=[]
countA = Counter()
for line in open('sentiment.txt'):
    countA += Counter(line.split(" ")[1:])

In [152]:
stop_word=[]
for i,j in countA.most_common(30):
    stop_word.append(i)
print("STOP WORD: ",stop_word)

STOP WORD:  ['.', '\n', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'that', 'it', 'as', 'but', 'with', 'film', 'for', 'this', 'its', 'an', 'movie', "it's", 'be', 'on', 'you', 'not', 'by', 'about', 'one', 'more']


### (´･ω･｀)< sentiment.txtの頻度を調べてみた. "movie"とか頻出に入ってしまった(；´∀｀)

In [154]:
def include_stopword(word,stop_word):
    if word in stop_word:
        return True
    else:
        return False

In [155]:
include_stopword("Hello",stop_word)

False

In [160]:
include_stopword("not",stop_word)

True

>ストップワード(stopword)  
文書中の ”the”, ”is”, “have”, “take”など、
話題の種類と関連を持たないと考えられる語。
ストップワードをあらかじめ削除してからベクトル化を行うことが多い。

## 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

>ステミング(stemming)  
同じような話題を指している語を一つの素性としてみなしたいとき、
派生語なども含めて同一の素性とみなす作業のこと。
例. ”run”, “runs”, “ran” さらに”runner”など。

>ポーターのステマー(Prter's stemmer)  
英語のステミングの手法. 多くの規則がある.  
規則例
- 語尾のedを除去する(walked→walk)
- 語尾のateを除去する(passionate→passion)
- 語尾のationalを除去する(operational→oper)
  
>“operational”と ”operate”は同じ ”oper” として扱えるが、
(hundred→hundr)となってしまったり、
(international→intern)と、意味が変わってしまうこともある。

In [162]:
from collections import defaultdict